Crawler para extração dos itens do carrinho da Atividade Prática.

# Imports

In [2]:
import requests, json

In [3]:
headers = {
    "Content-Type": "application/json",
    "Accept": "application/json",
}

payload = {
    "partner": "linx",
    "page": 1,
    "resultsPerPage": 300,
    "multiCategory": "alimentos",
    "sortBy": "relevance",
    "department": "ecom",
    "storeId": 461,
    "customerPlus": True
}

response = requests.post(r"https://api.vendas.gpa.digital/pa/search/category-page", headers=headers, data=json.dumps(payload))

max_produtos = response.json()['totalProducts']
payload['resultsPerPage'] = max_produtos

response = requests.post(r"https://api.vendas.gpa.digital/pa/search/category-page", headers=headers, data=json.dumps(payload))
lista_p = list(response.json()['products'])
lista_cod = []

for prod in lista_p:
    lista_cod.append(prod['id'])


In [4]:
len(lista_cod)

100

In [ ]:
texto = requests.get(r"https://www.paodeacucar.com/produto/86559").text
texto

In [ ]:
# Encontrando as posições
inicio = texto.find('"props":') + len('"props":')
fim = texto.find(',"page":"/produto"', inicio)

# Extraindo a substring
substring = texto[inicio:fim].strip()

substring

In [ ]:
# Transformando em dicionário
dicionario = json.loads(substring)
dicionario

In [ ]:
dicionario['initialProps']['componentProps']['product'].keys()

In [ ]:
dicionario['initialProps']['componentProps']['product']['brand']

In [ ]:
dicionario['initialProps']['componentProps']['product']['attributeGroups'][2]['attributes']

In [40]:
def get_product_info(code: int, verbose: bool = False) -> dict:
    try:
        # Fazendo a requisição
        texto = requests.get(fr"https://www.paodeacucar.com/produto/{code}").text
        
        # Encontrando as posições
        inicio = texto.find('"props":') + len('"props":')
        fim = texto.find(',"page":"/produto"', inicio)

        # Extraindo a substring
        substring = texto[inicio:fim].strip()
        
        # Dicionario Geral
        dicionario = json.loads(substring)
        
        # Informações gerais do produto
        informacoes = dicionario['initialProps']['componentProps']['product']
        
        # Dicionário com os resultados
        resultados = {    
        'id': informacoes['id'], 
        'nome': informacoes['name'], 
        'sku': informacoes['sku'], 
        'descricao': informacoes['description'],
        'imagem': f'https://static.paodeacucar.com{informacoes['thumbPath']}'
        }
        
        informacoes2 = informacoes['attributeGroups'][2]['attributes']
        
        # Códigos dos dados que queremos extrair
        codes_desejados = ['Marca', 'alturaProduto', 'larguraProduto', 'pesoBruto', 'unidadePesoBruto', 'profundidadeProduto']

        # Extraindo os valores
        resultados2 = {code: next((item['value'] for item in informacoes2 if item['code'] == code), None) for code in codes_desejados}
        
        # Adicionando ao dicionário
        final = {**resultados, **resultados2}
        
        assert (
            final['id'] == code and 
            final['alturaProduto'] != None and
            final['larguraProduto'] != None and
            final['pesoBruto'] != None and
            final['unidadePesoBruto'] != None and
            final['profundidadeProduto'] != None
        )

    except Exception as e:
        if verbose:
            print(f"Erro ao extrair informações do produto {code}!\nErro:\n{e}")
            
            if 'informacoes2' in locals():
                print(locals()['informacoes2'])
            elif 'informacoes' in locals():
                print(locals()['informacoes'])
            
        return None
    
    else:
        return final

In [42]:
i = 0
for code in lista_cod:
    prod = get_product_info(code)
    if prod:
        i+=1
        print(prod)
        print('-'*50)

print(i)

{'id': 86559, 'nome': 'Azeite Português de Oliva Extra Virgem ANDORINHA Vidro 500ml', 'sku': '0669252', 'descricao': '<b> Ingredientes:</b><br> Azeite de oliva extra virgem.<br><b>NÃO CONTÉM GLÚTEN.</b><br><b>Acidez:</b><br>0,50%.', 'imagem': 'https://static.paodeacucar.com/img/uploads/1/239/673239.png', 'Marca': 'Andorinha', 'alturaProduto': '25.7', 'larguraProduto': '6.6', 'pesoBruto': '837', 'unidadePesoBruto': 'G', 'profundidadeProduto': '6.6'}
--------------------------------------------------
{'id': 34660, 'nome': 'Brócolis Ninja QUALITÁ 300g', 'sku': '3007808', 'descricao': 'Fonte de fibras, contém ferro, potássio, cálcio e magnésio. Também possui vitaminas A, C, E e K.<br><b>Dica de consumo:</b>\r\nConsumo como saladas, sopas, cremes, composição de pratos. <br><b><br><b>NÃO CONTÉM GLÚTEN.</b>', 'imagem': 'https://static.paodeacucar.com/img/uploads/1/386/24783386.jpg', 'Marca': 'Qualitá', 'alturaProduto': '10.3', 'larguraProduto': '15.3', 'pesoBruto': '412', 'unidadePesoBruto': 

In [37]:
get_product_info(86559)

{'id': 86559,
 'nome': 'Azeite Português de Oliva Extra Virgem ANDORINHA Vidro 500ml',
 'sku': '0669252',
 'descricao': '<b> Ingredientes:</b><br> Azeite de oliva extra virgem.<br><b>NÃO CONTÉM GLÚTEN.</b><br><b>Acidez:</b><br>0,50%.',
 'imagem': 'https://static.paodeacucar.com/img/uploads/1/239/673239.png',
 'Marca': 'Andorinha',
 'alturaProduto': '25.7',
 'larguraProduto': '6.6',
 'pesoBruto': '837',
 'unidadePesoBruto': 'G',
 'profundidadeProduto': '6.6'}